In [1]:
import pickle
import pandas as pd
import re

with open('data_pickle_format/synethetictotrain.pickle', 'rb') as f:
    synethetic = pickle.load(f)
with open('data_pickle_format/labeledtotrain.pickle', 'rb') as f:
    labeled = pickle.load(f)
with open('data_pickle_format/unlabeled_vacancies_final.pickle', 'rb') as f:
    unlabeled = pickle.load(f)

In [2]:
from string import punctuation
import html
import re

def preprocess_text(text):
    text = text.lower()
    text = html.unescape(text)
    text = re.sub(r"https?://\S+", "", text) # remove http links
    text = re.sub(r"<.*?>", " ", text) # remove html tags
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    return text

unlabeled["job_description"] = unlabeled["job_description"].map(preprocess_text)

pd.set_option('display.max_colwidth', None)

In [3]:
training = pd.concat([synethetic, labeled], axis=0)

In [4]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

train_df = training[["job_description", "women_proportion"]].dropna()
train_df.columns = ["text", "labels"]

from sklearn.model_selection import train_test_split
train_df, dev_df = train_test_split(train_df, test_size=0.1, random_state=42)


/home/jkoz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()
model_args.num_train_epochs = 4  # Reduce training time
model_args.train_batch_size = 8  # Lower memory usage
model_args.eval_batch_size = 8   # Lower memory usage
model_args.max_seq_length = 128  # Limit input size
model_args.no_cache = True       # Avoid writing to disk
model_args.no_save = True        # Don’t save model files
model_args.overwrite_output_dir = True 
model_args.evaluate_during_training = False  # Speeds things up
model_args.regression = True  # Because your labels are floats


model = ClassificationModel(
    "distilbert",
    "distilbert-base-multilingual-cased",
    num_labels=1,
    args=model_args,
    use_cuda=torch.cuda.is_available(),
)

model.train_model(train_df[['text', 'labels']].sample(500), eval_df=dev_df[['text', 'labels']].sample(100))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 of 4:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
preds, raw_outputs = model.predict(unlabeled['job_description'].tolist()[:30])

1it [00:00,  5.34it/s]
100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


In [ ]:
sample = unlabeled.iloc[:30].copy()
sample["predicted_women_proportion"] = preds
sample.head(30)

positiontitle  \
0                  Freelance Technisch Consultant   
1                            Technisch Consultant   
2                                Support Engineer   
3                           Zelfstandig Architect   
4    Java Solution Architect / Software Architect   
5                              Software Architect   
9   Senior Consultant SAP Supply Chain Management   
10                    Manager SAP S/4hana Finance   
11                                   C Consultant   
30                            Wordpress Developer   
31                            Wordpress Developer   
32                        Wordpress Web Developer   
42              Sitecore Developer (Senior / Lead   
43                        Lead Developer Sitecore   
44              Sitecore Developer (Senior / Lead   
45      Scriptiestage Digital Workflow Management   
46                     SAP Integration Consultant   
47                  SAP Retail/Fashion Consultant   
48              Microsoft Dynamics CRM Consultant   
49                            Xpertdoc Consultant   
50  Microsoft Dynamics F&O Functioneel Consultant   
57                        Front-end Web Developer   
58                     Senior Application Manager   
59                      Software Engineer Analyst   
72                  Social Deal Android Developer   
73                  Ontwikkelaar Duurzame Energie   
74                         Frontend Developer Job   
90       Lead Functional SAP Consultant Logistics   
91                             SAP EWM Consultant   
92  Senior Consultant SAP Supply Chain Management   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sample_true = dev_df['labels'].values[:30]
sample_preds, _ = model.predict(dev_df['text'].tolist()[:30])

mse = mean_squared_error(sample_true, sample_preds)
mae = mean_absolute_error(sample_true, sample_preds)
r2 = r2_score(sample_true, sample_preds)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


1it [00:00,  5.01it/s]
100%|██████████| 1/1 [00:01<00:00,  1.37s/it]

Mean Squared Error: 0.06578182445892719
Mean Absolute Error: 0.22318149023369135
R^2 Score: -0.025856795109892472
